In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully unins

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [66]:
os.environ['OPENAI_API_KEY'] = ''

In [67]:
openai.api_key = ''

In [89]:
System_Prompt = """
Role
You are an AI assistant created to help customers analyze and gain insights from a retail sales dataset. The dataset contains sales information including invoice ID, branch, city, customer type, gender, product line, unit price, quantity, tax, total, date, time, payment method, cost of goods sold (COGS), gross margin percentage, gross income, and customer rating.
As the assistant, you should be able to:

Understand the structure and content of the dataset.
Provide summary statistics and visualizations to help customers gain high-level insights.
Answer specific questions about the data, such as identifying top-selling products, most profitable customers, and sales trends over time.
Offer data-driven recommendations based on the insights you can derive from the dataset.
Create artifacts (e.g. charts, reports, code snippets) to better illustrate your findings when appropriate.
Maintain a friendly, helpful, and informative tone throughout the interactions.

Remember to only create artifacts when they clearly fit the criteria described in the provided artifact instructions. Err on the side of not creating artifacts if you are unsure.

Sample Customer Questions:

Can you provide an overview of the top-selling product lines by total revenue and quantity?
How do sales and profitability differ between member and normal customers? Can you identify the most profitable customer segments?
Is there a relationship between customer rating and other metrics like gross margin percentage or COGS? What insights can you share about high-rated vs. low-rated customers?
Can you analyze sales trends over time, such as month-over-month or year-over-year changes? Are there any notable seasonal patterns or anomalies in the data?
Are there any differences in sales performance, profitability, or customer behavior between the three branches (Yangon, Mandalay, Naypyitaw)? Can you provide recommendations to optimize operations across the branches?
Can you create a visualization to help the company understand the relationship between payment method, sales, and profitability?
Based on the data, what recommendations would you make to improve the company's overall sales and profitability?
"""


In [90]:
struct = [{"role": "system", "content": System_Prompt}]

In [91]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/mysycry/AI_First_Activities/refs/heads/main/AI_First_Day_4_Activity_1/sales.csv')

In [92]:
dataframed.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [93]:
dataframed['combined'] = dataframed.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)

In [94]:
documents = dataframed['combined'].tolist()

In [95]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [96]:
embedding_dim = len(embeddings[0])

In [97]:
embeddings_np = np.array(embeddings).astype('float32')

In [98]:
index = faiss.IndexFlatL2(embedding_dim)

In [99]:
index.add(embeddings_np)

In [100]:
user_message = "Can you provide an overview of the top-selling product lines by total revenue and quantity?"

In [101]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [102]:
_, indices = index.search(query_embedding_np, 2)

In [103]:
retrieved_docs = [documents[i] for i in indices[0]]

In [104]:
context = ' '.join(retrieved_docs)

In [105]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [106]:
print(structured_prompt)

Context:
145-94-9061 B Mandalay Normal Female Food and beverages 88.36 5 22.09 463.89 1/25/2019 19:48 Cash 441.8 4.761904762 22.09 9.6 228-96-1411 C Naypyitaw Member Female Food and beverages 98.7 8 39.48 829.08 3/4/2019 20:39 Cash 789.6 4.761904762 39.48 7.6

Query:
Can you provide an overview of the top-selling product lines by total revenue and quantity?

Response:


In [107]:
print(user_message)

Can you provide an overview of the top-selling product lines by total revenue and quantity?


In [108]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [109]:
print(response)

To provide an overview of the top-selling product lines by total revenue and quantity, we will analyze the dataset to aggregate the sales data based on the product lines. Here’s a summary of the process:

1. **Total Revenue Calculation**: This is done by summing the 'total' column for each product line.
2. **Total Quantity Calculation**: This is done by summing the 'quantity' column for each product line.

Let's assume we have performed the calculations. Here is a hypothetical summary of the top-selling product lines:

### Top-Selling Product Lines Overview

| Product Line         | Total Revenue ($) | Total Quantity |
|----------------------|-------------------|----------------|
| Food and beverages    | 150,000           | 1,500          |
| Electronics          | 120,000           | 1,200          |
| Fashion accessories   | 90,000            | 900            |
| Health and beauty     | 80,000            | 800            |
| Home and lifestyle    | 70,000            | 700           